In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import joblib

In [2]:
#Load required Datasets
dataset1=pd.read_csv("dataset1.csv")
dataset2=pd.read_csv("dataset2.csv")
dataset=dataset2.copy()

In [3]:
#Importing and testing custom spell checker based on Norvig and Contraction Expansion
from NorvigSpellCheck import sentenceCorrection
from acronymsExpansion import  expandContractions
sentenceCorrection(expandContractions("is SOP for ntwrk errpr available"))

'is standard operating procedure for network error available'

In [4]:
import string
def bpe_clean(s):
#    s=sentenceCorrection(expandContractions(s))
    s=str(s).lower()
    for punctuation in string.punctuation:
        s=s.replace(punctuation," ")
    for n in "123456789":
        s=s.replace(n,"0")
    s=s.lower().split()
    s=[n for n in s if n.isalnum()==True]
    s=" ".join(s)
    s=s.encode("ascii","ignore").decode("ascii")
    return s



In [4]:
stop_words=stopwords.words('english')
stop_words=stop_words+['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
stop_words=stop_words+["emailidx",'sap','please','dear','regards','kind','best','help','team','ticket','hi','thank','phonex','thanks','maple','leaf','cc','to','subject','mapleleaf','sent','hello','httpx','email','food','foods']
def clean_tfidf(ticketText):
    ticketText=bpe_clean(ticketText)
    #stop_words=stopwords.words('english')
    words=nltk.tokenize.word_tokenize(ticketText)
    words=[n for n in words if n not in stop_words]
    words=[n for n in words if n.isalpha()]
    return " ".join(words)

In [5]:
#upsample minority classes
from sklearn.utils import resample
def Resample(X_train,y_train):
    dfR=pd.DataFrame(X_train)
    dfR["assignment_group"]=y_train
    sampleData=dfR.groupby("assignment_group").description.count().reset_index().sort_values("description",ascending=True)
    majority_class=sampleData.assignment_group.values[-1]
    minority_classes=list(sampleData.assignment_group.unique())
    minority_classes.remove(majority_class)
    minority_classes

    dfResampled=dfR[dfR.assignment_group==majority_class]
    n_samples=len(dfResampled)
    for minority_class in minority_classes:
#        print(minority_class)
        dfMinority=dfR[dfR.assignment_group==minority_class]
        dfMinorityUpSample= resample(dfMinority,replace=True,random_state=42,n_samples=n_samples)
        dfResampled=pd.concat([dfResampled,dfMinorityUpSample])
    X_train=dfResampled.description
    y_train=dfResampled.assignment_group
    return X_train,y_train
    print(dfResampled.assignment_group.unique())
    print(len(dfResampled))


In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

In [7]:
#Train Test Split

df2=dataset.copy()
df2["description"]=df2.apply(lambda x: clean_tfidf(x.description),axis=1)

X_train, X_test, y_train, y_test = train_test_split(df2.description, df2.assignment_group, test_size=0.20, random_state=42)#,stratify=df.assignment_group)
X_train,y_train=Resample(X_train,y_train)


In [8]:
import keras 
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Activation, MaxPooling1D, Flatten,MaxPooling2D, Embedding, MaxPooling1D,Conv1D
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from keras import utils
from keras.preprocessing import sequence
from keras.preprocessing import text
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score
from keras.callbacks import EarlyStopping

In [9]:
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
import tracemalloc,time
tracemalloc.start()
startTime=time.time()

encoder = LabelEncoder()
encoder.fit(y_train)
y_train_enc=encoder.transform(y_train)
y_train_enc=utils.to_categorical(y_train_enc)

y_test_enc=encoder.transform(y_test)
y_test_enc=utils.to_categorical(y_test_enc)

In [10]:
tokenizer = text.Tokenizer(num_words=1000)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

X_train = sequence.pad_sequences(X_train, maxlen=1000)
X_test = sequence.pad_sequences(X_test, maxlen=1000)

In [11]:
num_classes=len(y_train.unique())
model = Sequential()
model.add(Embedding(1000,100,input_length=1000))
model.add(Conv1D(32,1,padding="valid",activation="relu"))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(num_classes,activation="softmax"))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 100)         100000    
_________________________________________________________________
conv1d (Conv1D)              (None, 1000, 32)          3232      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 500, 32)           0         
_________________________________________________________________
flatten (Flatten)            (None, 16000)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                160010    
Total params: 263,242
Trainable params: 263,242
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
ES=EarlyStopping(monitor='val_accuracy',min_delta=0, patience=10,verbose=0, mode="auto",baseline=None)
callbacks_list=[ES]

In [13]:
model.fit(X_train,y_train_enc,validation_data=(X_test,y_test_enc),batch_size=32,epochs=40,callbacks=callbacks_list)
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
stopTime=time.time()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
print(stopTime-startTime)



Epoch 1/40
459/459 [==============================] - 23s 51ms/step - loss: 2.2935 - accuracy: 0.1267 - val_loss: 2.2891 - val_accuracy: 0.1584
Epoch 2/40
459/459 [==============================] - 31s 68ms/step - loss: 2.2394 - accuracy: 0.1698 - val_loss: 2.2417 - val_accuracy: 0.1120
Epoch 3/40
459/459 [==============================] - 25s 55ms/step - loss: 2.1884 - accuracy: 0.1995 - val_loss: 2.1481 - val_accuracy: 0.2730
Epoch 4/40
459/459 [==============================] - 23s 51ms/step - loss: 2.1503 - accuracy: 0.2281 - val_loss: 2.1916 - val_accuracy: 0.1651
Epoch 5/40
459/459 [==============================] - 23s 51ms/step - loss: 2.1107 - accuracy: 0.2384 - val_loss: 2.1726 - val_accuracy: 0.1516
Epoch 6/40
459/459 [==============================] - 23s 49ms/step - loss: 2.0664 - accuracy: 0.2494 - val_loss: 2.2112 - val_accuracy: 0.1719
Epoch 7/40
459/459 [==============================] - 23s 51ms/step - loss: 2.0231 - accuracy: 0.2641 - val_loss: 2.1217 - val_accuracy: